# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 1:39PM,246887,20,8552102-T,"ACI Healthcare USA, Inc.",Released
1,Sep 21 2022 1:37PM,246963,10,8555908,Yusen – 3D Matrix,Released
2,Sep 21 2022 1:36PM,246962,19,60017411,"GUSA Granules USA, Inc.",Released
3,Sep 21 2022 1:31PM,246961,20,6995896,Else Nutrition,Released
4,Sep 21 2022 1:30PM,246960,19,60017420,"GUSA Granules USA, Inc.",Released
5,Sep 21 2022 1:28PM,246959,10,CTF0011422,"Citieffe, Inc.",Released
6,Sep 21 2022 1:28PM,246959,10,CTF0011423,"Citieffe, Inc.",Released
7,Sep 21 2022 1:25PM,246957,10,8555902,Yusen – 3D Matrix,Released
8,Sep 21 2022 1:20PM,246924,20,ALU0007979,Alumier Labs Inc.,Released
9,Sep 21 2022 1:20PM,246955,19,ACG-2102491313,ACG North America LLC,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,246959,Released,2
41,246960,Released,1
42,246961,Released,1
43,246962,Released,1
44,246963,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246959,NaN,NaN,2.0
246960,NaN,NaN,1.0
246961,NaN,NaN,1.0
246962,NaN,NaN,1.0
246963,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246864,5.0,0.0,1.0
246871,0.0,2.0,0.0
246872,0.0,0.0,1.0
246878,0.0,1.0,0.0
246887,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246864,5,0,1
246871,0,2,0
246872,0,0,1
246878,0,1,0
246887,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246864,5,0,1
1,246871,0,2,0
2,246872,0,0,1
3,246878,0,1,0
4,246887,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246864,5,,1
1,246871,,2,
2,246872,,,1
3,246878,,1,
4,246887,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 1:39PM,246887,20,"ACI Healthcare USA, Inc."
1,Sep 21 2022 1:37PM,246963,10,Yusen – 3D Matrix
2,Sep 21 2022 1:36PM,246962,19,"GUSA Granules USA, Inc."
3,Sep 21 2022 1:31PM,246961,20,Else Nutrition
4,Sep 21 2022 1:30PM,246960,19,"GUSA Granules USA, Inc."
5,Sep 21 2022 1:28PM,246959,10,"Citieffe, Inc."
7,Sep 21 2022 1:25PM,246957,10,Yusen – 3D Matrix
8,Sep 21 2022 1:20PM,246924,20,Alumier Labs Inc.
9,Sep 21 2022 1:20PM,246955,19,ACG North America LLC
10,Sep 21 2022 1:11PM,246953,19,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 1:39PM,246887,20,"ACI Healthcare USA, Inc.",,,1
1,Sep 21 2022 1:37PM,246963,10,Yusen – 3D Matrix,,,1
2,Sep 21 2022 1:36PM,246962,19,"GUSA Granules USA, Inc.",,,1
3,Sep 21 2022 1:31PM,246961,20,Else Nutrition,,,1
4,Sep 21 2022 1:30PM,246960,19,"GUSA Granules USA, Inc.",,,1
5,Sep 21 2022 1:28PM,246959,10,"Citieffe, Inc.",,,2
6,Sep 21 2022 1:25PM,246957,10,Yusen – 3D Matrix,,,1
7,Sep 21 2022 1:20PM,246924,20,Alumier Labs Inc.,,,1
8,Sep 21 2022 1:20PM,246955,19,ACG North America LLC,,1,
9,Sep 21 2022 1:11PM,246953,19,Else Nutrition,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 1:39PM,246887,20,"ACI Healthcare USA, Inc.",1,,
1,Sep 21 2022 1:37PM,246963,10,Yusen – 3D Matrix,1,,
2,Sep 21 2022 1:36PM,246962,19,"GUSA Granules USA, Inc.",1,,
3,Sep 21 2022 1:31PM,246961,20,Else Nutrition,1,,
4,Sep 21 2022 1:30PM,246960,19,"GUSA Granules USA, Inc.",1,,
5,Sep 21 2022 1:28PM,246959,10,"Citieffe, Inc.",2,,
6,Sep 21 2022 1:25PM,246957,10,Yusen – 3D Matrix,1,,
7,Sep 21 2022 1:20PM,246924,20,Alumier Labs Inc.,1,,
8,Sep 21 2022 1:20PM,246955,19,ACG North America LLC,,1,
9,Sep 21 2022 1:11PM,246953,19,Else Nutrition,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 1:39PM,246887,20,"ACI Healthcare USA, Inc.",1,,
1,Sep 21 2022 1:37PM,246963,10,Yusen – 3D Matrix,1,,
2,Sep 21 2022 1:36PM,246962,19,"GUSA Granules USA, Inc.",1,,
3,Sep 21 2022 1:31PM,246961,20,Else Nutrition,1,,
4,Sep 21 2022 1:30PM,246960,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 1:39PM,246887,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
1,Sep 21 2022 1:37PM,246963,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Sep 21 2022 1:36PM,246962,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Sep 21 2022 1:31PM,246961,20,Else Nutrition,1.0,NaN,NaN
4,Sep 21 2022 1:30PM,246960,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 1:39PM,246887,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
1,Sep 21 2022 1:37PM,246963,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Sep 21 2022 1:36PM,246962,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Sep 21 2022 1:31PM,246961,20,Else Nutrition,1.0,0.0,0.0
4,Sep 21 2022 1:30PM,246960,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4197877,68.0,12.0,7.0
12,493811,2.0,0.0,0.0
15,493855,10.0,0.0,18.0
16,493779,0.0,7.0,0.0
19,1481705,10.0,17.0,0.0
20,1481532,28.0,8.0,0.0
21,246878,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4197877,68.0,12.0,7.0
1,12,493811,2.0,0.0,0.0
2,15,493855,10.0,0.0,18.0
3,16,493779,0.0,7.0,0.0
4,19,1481705,10.0,17.0,0.0
5,20,1481532,28.0,8.0,0.0
6,21,246878,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,68.0,12.0,7.0
1,12,2.0,0.0,0.0
2,15,10.0,0.0,18.0
3,16,0.0,7.0,0.0
4,19,10.0,17.0,0.0
5,20,28.0,8.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,68.0
1,12,Released,2.0
2,15,Released,10.0
3,16,Released,0.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,7.0,0.0,18.0,0.0,0.0,0.0,0.0
Executing,12.0,0.0,0.0,7.0,17.0,8.0,1.0
Released,68.0,2.0,10.0,0.0,10.0,28.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,7.0,0.0,18.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,0.0,7.0,17.0,8.0,1.0
2,Released,68.0,2.0,10.0,0.0,10.0,28.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,7.0,0.0,18.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,0.0,7.0,17.0,8.0,1.0
2,Released,68.0,2.0,10.0,0.0,10.0,28.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()